# Monte-Carlo - Convex Hull

This program tries to find several 100 points that are the sup norm, and then use convex hull to find the convex hull.
This has too many dimensions and crashes almost instantly.

In [4]:
import numpy as np
from scipy.optimize import linprog


def montecarlo(array, b_desired):

    # Helper functions
    def initialize_vector():
        k = np.random.randint(5, 20) # Change
        initial_phi = np.zeros(len(array[0]))
        random_indices = np.random.choice(len(initial_phi), k, replace=False)
        numbers = [np.random.uniform(0, 0.1)]
        random_max = 0.05  
        for _ in range(k - 1):
            numbers.append(np.random.uniform(0, random_max - np.sum(numbers)))
        initial_phi[random_indices] = numbers
        return initial_phi

    def calculate_error(phi, penal_mult):
        computed_b = np.dot(array, phi)
        diff = b_desired - computed_b
        penalty1 = -100 * np.sum(diff[diff < 0])
        penalty2 = 0  # .1 * np.sum(diff[diff > 0])  # L1 penalty (currently unused)
        penalty3 = 2 * np.max(diff)
        penalty4 = penal_mult * (1 - sum(phi)) ** 2
        penalties = penalty1 + penalty3 + penalty4 + penalty2
        return penalties

    # Parameters
    T = 10e-4
    T_now = T
    T_min = 10e-8
    alpha = 0.42
    num_iterations = 40_000
    penal_mult = 0.25
    step_size = 0.03

    # Initializations
    iter = 0
    counter = []
    error_values = []
    best_values_all = []
    best_phi = np.zeros(len(array[0]))
    best_error = np.inf
    current_phi = np.zeros(len(array[0]))#initialize_vector()
    current_error = np.inf

    # Monte Carlo simulation
    while T > T_min:
        
        if T_now > T:
            #print('T',np.round(T,7))
            T_now = T

        for _ in range(num_iterations):
            iter += 1

            # Perturb phi
            new_phi = current_phi.copy()
            index = np.random.randint(0, len(new_phi))
            change = np.random.uniform(-step_size, step_size)
            new_phi[index] += change
            new_phi[new_phi < 0] = 0


            # Skip if sum constraint is violated
            if np.sum(new_phi) > 1:
                continue

            # Calculate new error
            new_error = calculate_error(new_phi, penal_mult)

            # Acceptance criterion
            if new_error < current_error or np.random.uniform() < np.exp((current_error - new_error) / T):
                current_phi = new_phi
                current_error = new_error
                counter.append(iter)
                error_values.append(current_error / 2)
                if new_error < best_error:
                    best_error = new_error
                    best_phi = new_phi
                    best_values_all.append(best_phi)

        # Cooling schedule
        T *= alpha
        step_size *= alpha**(0.5)
        penal_mult *= 1.01

    # Visualization (optional)
    #plt.plot(counter[600:], error_values[600:])
    #plt.grid()
    #plt.show()

    # Results
    b_found = array.dot(best_phi)
    score = b_desired - b_found

    # Print results (optional)
    print('b desired', np.round(b_desired,4))
    print('b found', np.round(b_found, 4))
    print()
    print('difference', np.round(score, 4))
    print("Sup Norm is", np.max(score))
    print("L2 norm is", np.linalg.norm(score, 2))
    print()
    print('best_phi',np.round(best_phi,3))

    return counter, error_values, best_phi, b_found, score



In [5]:
import numpy as np

file_path = "BNI-simexp.dat"
A_matrix = np.zeros((100, 10))  # Initialize a 100x10 matrix of zeros
count_matrix = np.zeros((100, 10))  # To keep track of how many values go into each index for averaging

with open(file_path, 'r') as file:
    for i, line in enumerate(file):
        if i==2:
            b_array = line.split()
            b_array = np.array(b_array)
            b_array = b_array.astype(np.float64)
        
        if i >= 3 and i <= 1002:  # Skip the first three lines, process lines 4 to 1002
            data = line.split()
            row = (i - 3) // 10  # Determine the current row in A_matrix
            col_index = int(data[2]) - 1  # The 3rd column specifies the index (subtract 1 for 0-based indexing)

            if len(data) >= 5:  # Ensure there are at least 5 elements
                fifth_value = float(data[4])  # Get the 5th column value

                # Update A_matrix by averaging if there are multiple values targeting the same index
                A_matrix[row, col_index] += fifth_value
                count_matrix[row, col_index] += 1  # Increment count for averaging

# Average the values where applicable
A_matrix = np.divide(A_matrix, count_matrix, out=np.zeros_like(A_matrix), where=count_matrix != 0)

# Print the rounded matrix
print(b_array)
print()
print(np.round(A_matrix, 3))


[0.007    0.001    0.025    0.034    0.008553 0.2265   0.002446 0.007
 0.154    0.01038 ]

[[0.    0.    0.032 0.106 0.    0.701 0.028 0.    0.616 0.   ]
 [0.    0.598 0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.026 0.    0.    0.028 0.    0.    0.    0.508 0.   ]
 [0.108 0.    0.536 0.555 0.    0.    0.026 0.    0.689 0.   ]
 [0.    0.    0.    0.    0.036 0.667 0.028 0.    0.616 0.026]
 [0.026 0.    0.    0.179 0.    0.    0.    0.62  0.034 0.   ]
 [0.    0.026 0.    0.116 0.    0.    0.    0.551 0.    0.   ]
 [0.    0.026 0.519 0.616 0.    0.    0.    0.    0.    0.   ]
 [0.628 0.    0.    0.    0.    0.    0.    0.026 0.    0.   ]
 [0.    0.    0.472 0.    0.034 0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.61  0.    0.   ]
 [0.    0.    0.    0.    0.028 0.554 0.    0.    0.324 0.034]
 [0.    0.    0.    0.534 0.    0.    0.    0.    0.179 0.   ]
 [0.    0.    0.    0.028 0.    0.645 0.249 0.    0.36  0.   ]
 [0.    0.026 0.    0.    0

In [6]:
A= A_matrix
b = b_array

all_best_vs = []
all_best_sups = []
for a in range (0,1000):
    counter, error_values, best_phi, b_found, score = montecarlo(A.T, b)
    all_best_sups.append(np.max(score))
    all_best_vs.append(best_phi)
    print(a)
print(np.round(all_best_sups,3))

b desired [0.007  0.001  0.025  0.034  0.0086 0.2265 0.0024 0.007  0.154  0.0104]
b found [0.007  0.001  0.025  0.0338 0.0084 0.1991 0.0024 0.007  0.154  0.0103]

difference [0.     0.     0.     0.0002 0.0001 0.0274 0.     0.     0.     0.    ]
Sup Norm is 0.027361361171984844
L2 norm is 0.027362193517982157

best_phi [0.037 0.    0.    0.    0.021 0.    0.    0.    0.007 0.044 0.    0.121
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.001 0.    0.    0.    0.    0.    0.    0.    0.    0.003 0.    0.
 0.    0.    0.    0.    0.    0.002 0.003 0.002 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.003 0.    0.    0.017
 0.    0.    0.074 0.    0.    0.    0.    0.035 0.    0.    0.018 0.
 0.    0.    0.    0.    0.    0.    0.011 0.036 0.    0.006 0.034 0.004
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.   ]
0
b desired [0.007  0.001  0.025  0.034  0.0086 0.2265 0.0024 0.007  0.154  0.0104]


In [12]:
import numpy as np

# Convert to numpy arrays for easier manipulation
all_best_sups = np.array(all_best_sups)
all_best_vs = np.array(all_best_vs)

# Get indices where sup is less than 0.001
indices_below_threshold = np.where(all_best_sups < 0.01)[0]

# Extract the corresponding best_phi values from all_best_vs using these indices
all_best_vs2 = all_best_vs[indices_below_threshold]
all_best_sups2  = all_best_sups[indices_below_threshold]

# (Optional) Print or work with the results
print("The sup norm is", np.min(all_best_sups2))
print("we accept sup norms up to", np.max(all_best_sups2))
print("We are keeping this many points:", all_best_sups2.shape)



The sup norm is 4.985132319502983e-05
we accept sup norms up to 0.009817406333613554
We are keeping this many points: (134,)


In [22]:
from scipy.spatial import ConvexHull


# Compute the convex hull
hull = ConvexHull(all_best_vs2)

# Get the vertices of the convex hull
hull_vertices = hull.vertices

# Print the vertices indices
print("Vertices of the convex hull:", hull_vertices)

# Optionally, if you want to see the points corresponding to the vertices:
best_hull_points = all_best_sups2[hull_vertices]
print("Number of vertices in this approximate best hull area", best_hull_points.shape)
print("Points on the convex hull:", best_hull_points)


QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt Qx
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1762345289  incidence  Qtriangulate  Qxact-merge  _zero-centrum
  Q3-no-merge-vertices-dim-high  _max-width 0.19  Error-roundoff 3.4e-14
  _one-merge 6.8e-12  _near-inside 3.4e-11  Visible-distance 2e-13
  U-max-coplanar 2e-13  Width-outside 4.1e-13  _wide-facet 1.2e-12
  _maxoutside 6.9e-12

precision problems (corrected unless 'Q0' or an error)
    101 nearly singular or axis-parallel hyperplanes
    101 zero divisors during back substitute
  55978 zero divisors during gaussian elimination

The input to qhull appears to be less than 100 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p98(v101):     0     0     0     0 0.017     0     0     0 0.0074 0.037     0   0.1     0     0     0     0 0.00029     0     0     0 0.00

In [17]:
import numpy as np

for i in range(all_best_vs2.shape[1]):  # Iterate over columns
    if np.all(all_best_vs2[:, i] == all_best_vs2[0, i]):
        print(f"Dimension {i} has constant values")

Dimension 1 has constant values
Dimension 30 has constant values


In [19]:
variances = np.var(all_best_vs2, axis=0)
for i, variance in enumerate(variances):
    if variance < 1e-10:  # Set a very small threshold
        print(f"Dimension {i} has near-zero variance: {variance} and a value of", all_best_vs2[i,0])

Dimension 1 has near-zero variance: 0.0 and a value of 0.014880259027081764
Dimension 30 has near-zero variance: 0.0 and a value of 0.023720119745141047
Dimension 36 has near-zero variance: 9.840497846318437e-11 and a value of 0.0
Dimension 49 has near-zero variance: 1.0911438764242292e-11 and a value of 0.015433548094345058
Dimension 61 has near-zero variance: 2.7320272309825123e-13 and a value of 0.0
Dimension 93 has near-zero variance: 6.762060374551208e-11 and a value of 0.04989968296342143


In [24]:
import numpy as np
from scipy.spatial import ConvexHull

def remove_low_variance_dims(data, threshold=1e-5):

    variances = np.var(data, axis=0)
    keep_dims = variances > threshold
    return data[:, keep_dims]

# Apply the function to your data
best_of_best_filtered = remove_low_variance_dims(all_best_vs2)

# Now use best_of_best_filtered for ConvexHull
hull = ConvexHull(best_of_best_filtered)

QhullError: QH6235 qhull error (qh_memalloc): negative request size (-1475708936).  Did int overflow due to high-D?

While executing:  | qhull i Qt Qx
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1762799078  incidence  Qtriangulate  Qxact-merge  _zero-centrum
  Q3-no-merge-vertices-dim-high  _max-width 0.19  Error-roundoff 6.4e-15
  _one-merge 3.1e-13  _near-inside 1.6e-12  Visible-distance 3.8e-14
  U-max-coplanar 3.8e-14  Width-outside 7.7e-14  _wide-facet 2.3e-13
  _maxoutside 3.2e-13
Last point added to hull was p34.  Last merge was #80621.

At error exit:

Convex hull of 134 points in 24-d:

  Number of vertices: 39
  Number of facets: 12262622
  Number of non-simplicial facets: 3

Statistics for:  | qhull i Qt Qx

  Number of points processed: 38
  Number of hyperplanes created: 10485387
  Number of distance tests for qhull: 10541561
  Number of distance tests for merging: 283616027
  Number of distance tests for checking: 0
  Number of merged facets: 80621


precisio